# Data Augmentation

The goal of this notebook is to use data augmentation to improve upon the results of the original TorNet baseline CNN model.

In this notebook, we implement the image flipping used to only add flipped versions of tornado images to the TorNet dataset.
We then take that data and re-train the original architecture of the TorNet model (albeit with our re-tuned hyperparameters
for batch size and learning rate) in order to see whether the data augmentation strategy will improve the quality of the 
model that is trained by addressing the class imbalance due to the rarity of tornadoes in the dataset.

Of note - as mentioned in the project milestone, we can't use region cropping or translation here, because the 
original data is radar data which is being measured on a slant (not top-down) and hence has a natural stretching/distortion
that occurs depending on how far away a storm cell is from the center of the radar image. If we move the data around, 
we risk introducing our own squishing artifacts on the shapes of the cells which would poison our dataset. 

In comparison, mirroring here could be very helpful (without translating the data) to generate more examples of 
tornadoes without having to find more tornado radar images. Hence, we implement that strategy here.

In [1]:
import sys

import os
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

# just the location for the input data
TORNET_DATA_INPUT_FOLDER = "/mnt/c/users/handypark/Documents/Grad_School_Courses/CS_230/tornet"

2024-11-18 13:24:52.820643: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-18 13:24:52.872618: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731965092.904615   17130 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731965092.913038   17130 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-18 13:24:52.973035: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# Just making sure that we are indeed using GPU-based Tensorflow and not CPU-based Tensorflow.
tf.test.is_built_with_cuda()

# We tried experimental memory growth in some cases, but it didn't work out well (also crashed a lot).
"""
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
"""

'\ngpus = tf.config.list_physical_devices(\'GPU\')\nif gpus:\n  try:\n    # Currently, memory growth needs to be the same across GPUs\n    for gpu in gpus:\n      tf.config.experimental.set_memory_growth(gpu, True)\n    logical_gpus = tf.config.list_logical_devices(\'GPU\')\n    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")\n  except RuntimeError as e:\n    # Memory growth must be set before GPUs have been initialized\n    print(e)\n'

## Creating a dataset with additional mirrored tornado examples

This time around, now that we have the learning rate and batch size all tuned, we can start trying to improve the performance of the model.
Our first idea is to try using data augmentation. Since examples of tornados are rare (and comprise only a tiny portion of the data),
the hope here is to try to artificially increase the number of examples of tornadoes to help correct this class imbalance.

The initial paper used weights on different kinds of examples, but the hope here is that augmenting the data can be even more effective.
What we've done below is to take TorNet's dataset loading + creation code and introduce a flipping step, where we do the following:
- Check if the given training set image is a tornadic example.
- If not, proceed as normal.
- If so, add another copy of the image that we have flipped to the training set.

With this data augmentation, we're hoping to see some improvement in the performance of the model on the testing data in terms of AUC, just from the additional "examples" of tornados that we get a result.

In [3]:
"""
TorNet's data loading code, but now with some adjustments for data augmentation purposes.
We create `create_tf_with_flips_dataset` in order to make a dataset that has both
tornadic examples and mirrored tornadic examples.
"""
from typing import List, Dict

from tornet.data.loader import query_catalog, read_file
from tornet.data.constants import ALL_VARIABLES
from tornet.data import preprocess as pp
import numpy as np

def create_tf_with_flips_dataset(files:str,
                                 variables: List[str]=ALL_VARIABLES,
                                 n_frames:int=1,
                                 tilt_last: bool=True) -> tf.data.Dataset:
    """
    This is Tornet's main function for loading the data from the folder where it's all stored, but we've 
    made adjustments to actually add a system for flipping tornadic images.

    In this case, we add non-tornadic examples as normal, but when we encounter an example of
    a tornado, we actually add both the original tornadic example and a mirrored version
    of the tornadic example to the dataset.
    """
    assert len(files)>0
    
    # grab one file to gets keys, shapes, etc
    data = read_file(files[0],variables=variables,n_frames=n_frames, tilt_last=tilt_last)
    
    output_signature = { k:tf.TensorSpec(shape=data[k].shape,dtype=data[k].dtype,name=k) for k in data }

    # Iterate through all the files and keep track of which are tornadic examples, and add 
    # a "to-be-flipped" copy for each case of a tornado.
    file_list = []
    for f in files:
        file_list.append((f, 0))
        file_dict = read_file(f,variables=variables,n_frames=n_frames, tilt_last=tilt_last)
        if file_dict["label"] == 1:
            file_list.append((f, 1))

    # When generating the data, check to see whether we should flip the data,
    # then flip if necessary.
    def gen():
        for (f, flipper) in file_list:
            file_dict = read_file(f,variables=variables,n_frames=n_frames, tilt_last=tilt_last)
            if flipper == 1:
                # Each of the 6 different radar images gets flipped using np.flip
                for var in ['DBZ','VEL','KDP','RHOHV','ZDR','WIDTH']:
                    file_dict[var] = tf.convert_to_tensor(np.flip(file_dict[var], 1))
            yield file_dict
    ds = tf.data.Dataset.from_generator(gen,
                                        output_signature=output_signature)
    return ds
    

def preproc(ds: tf.data.Dataset,
            weights:Dict=None,
            include_az:bool=False,
            select_keys:list=None,
            tilt_last:bool=True):
    """
    This is Tornet's preprocessing function, unchanged,
    for taking the raw dataset loaded from the files (in create_tf_dataset)
    and then doing a few things:

    - Remove the time dimension (since we only care about detection at a given time t)
    - Add coordinates (so that we can run CoordConv layers later)
    - Split the data into its inputs and label outputs
    - Adding weights (if we decide to weight the data at all)

    Once the preprocessing is done, the data is basically ready to be trained on.
    """
    
    # Remove time dimension
    ds = ds.map(pp.remove_time_dim)

    # Add coordinate tensors
    ds = ds.map(lambda d: pp.add_coordinates(d,include_az=include_az,tilt_last=tilt_last,backend=tf))

    # split into X,y
    ds = ds.map(pp.split_x_y)

    # Add sample weights
    if weights:
        ds = ds.map(lambda x,y:  pp.compute_sample_weight(x,y,**weights, backend=tf) )
    
        # select keys for input
        if select_keys is not None:
            ds = ds.map(lambda x,y,w: (pp.select_keys(x,keys=select_keys),y,w))
    else:
        if select_keys is not None:
            ds = ds.map(lambda x,y: (pp.select_keys(x,keys=select_keys),y))

    return ds

In [4]:
def make_tf_with_flips_loader(data_root: str, 
                              data_type:str='train', # or 'test'
                              years: list=list(range(2013,2023)),
                              batch_size: int=128, 
                              weights: Dict=None,
                              include_az: bool=False,
                              random_state:int=1234,
                              select_keys: list=None,
                              tilt_last: bool=True,
                              from_tfds: bool=False,
                              tfds_data_version: str='1.1.0',
                              num_epochs: int=3):
    """
    We've trimmed down the make_tf_loader function from the TorNet helper
    functions to make clear which part of the loader we're using.

    Now that we have a function for making a dataset with flipped tornado
    examples added in, we make use of it here.
    """
    # get files from the catalog
    file_list = query_catalog(data_root, data_type, years, random_state)

    # make the dataset using the flipped data generator
    ds = create_tf_with_flips_dataset(file_list,variables=ALL_VARIABLES,n_frames=1, tilt_last=tilt_last) 

    # do the usual preprocessing to prepare the input for Tensorflow training
    ds = preproc(ds,weights,include_az,select_keys,tilt_last)
    ds = ds.prefetch(tf.data.AUTOTUNE)

    # this has been adjusted to include drop_remainder=True
    ds = ds.batch(batch_size, drop_remainder=True)
    return ds

## TorNet Baseline CNN Model Definition:

As stated in the comment for the code block below, this is just the TorNet model code that was used in the original paper.
We'll be using the same model for now during our data augmentation experiments.

In [5]:
"""
This is just the TorNet model code that was used in the paper.
Goal is to run this model which consists of:
- Normalizing the inputs
- Adding the coordinate information for CoordConv to work properly
- Running 4 VGG blocks which each have CoordConv2D and two MAXPOOL layers
- One last block of Conv2D layers and MAXPOOL to get the output probability
"""

from typing import Dict, List, Tuple
import numpy as np
import keras
from tornet.models.keras.layers import CoordConv2D, FillNaNs
from tornet.data.constants import CHANNEL_MIN_MAX, ALL_VARIABLES


def build_model(shape:Tuple[int]=(120,240,2),
                c_shape:Tuple[int]=(120,240,2),
                input_variables:List[str]=ALL_VARIABLES,
                start_filters:int=64,
                l2_reg:float=0.001,
                background_flag:float=-3.0,
                include_range_folded:bool=True,
                head='maxpool'):
    # Create input layers for each input_variables
    inputs = {}
    for v in input_variables:
        inputs[v]=keras.Input(shape,name=v)
    n_sweeps=shape[2]
    
    # Normalize inputs and concate along channel dim
    normalized_inputs=keras.layers.Concatenate(axis=-1,name='Concatenate1')(
        [normalize(inputs[v],v) for v in input_variables]
        )

    # Replace nan pixel with background flag
    normalized_inputs = FillNaNs(background_flag)(normalized_inputs)

    # Add channel for range folded gates 
    if include_range_folded:
        range_folded = keras.Input(shape[:2]+(n_sweeps,),name='range_folded_mask')
        inputs['range_folded_mask']=range_folded
        normalized_inputs = keras.layers.Concatenate(axis=-1,name='Concatenate2')(
               [normalized_inputs,range_folded])
        
    # Input coordinate information
    cin=keras.Input(c_shape,name='coordinates')
    inputs['coordinates']=cin

    x,c = normalized_inputs,cin
    
    x,c = vgg_block(x,c, filters=start_filters,   ksize=3, l2_reg=l2_reg, n_convs=2, drop_rate=0.1)   # (60,120)
    x,c = vgg_block(x,c, filters=2*start_filters, ksize=3, l2_reg=l2_reg, n_convs=2, drop_rate=0.1)  # (30,60)
    x,c = vgg_block(x,c, filters=4*start_filters, ksize=3, l2_reg=l2_reg, n_convs=3, drop_rate=0.1)  # (15,30)
    x,c = vgg_block(x,c, filters=8*start_filters, ksize=3, l2_reg=l2_reg, n_convs=3, drop_rate=0.1)  # (7,15)
    #x,c = vgg_block(x,c, filters=8*start_filters, ksize=3, l2_reg=l2_reg, n_convs=3)  # (3,7)
    
    if head=='mlp':
        # MLP head
        x = keras.layers.Flatten()(x) 
        x = keras.layers.Dense(units = 4096, activation ='relu')(x) 
        x = keras.layers.Dense(units = 2024, activation ='relu')(x) 
        output = keras.layers.Dense(1)(x)
    elif head=='maxpool':
        # Per gridcell
        x = keras.layers.Conv2D(filters=512, kernel_size=1,
                          kernel_regularizer=keras.regularizers.l2(l2_reg),
                          activation='relu')(x)
        x = keras.layers.Conv2D(filters=256, kernel_size=1,
                          kernel_regularizer=keras.regularizers.l2(l2_reg),
                          activation='relu')(x)
        x = keras.layers.Conv2D(filters=1, kernel_size=1,name='heatmap')(x)
        # Max in scene
        output = keras.layers.GlobalMaxPooling2D()(x)

    return keras.Model(inputs=inputs,outputs=output)


def vgg_block(x,c, filters=64, ksize=3, n_convs=2, l2_reg=1e-6, drop_rate=0.0):

    for _ in range(n_convs):
        x,c = CoordConv2D(filters=filters,
                          kernel_size=ksize,
                          kernel_regularizer=keras.regularizers.l2(l2_reg),
                          padding='same',
                          activation='relu')([x,c])
    x = keras.layers.MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
    c = keras.layers.MaxPool2D(pool_size =2, strides =2, padding ='same')(c)
    if drop_rate>0:
        x = keras.layers.Dropout(rate=drop_rate)(x)
    return x,c


def normalize(x,
              name:str):
    """
    Channel-wise normalization using known CHANNEL_MIN_MAX
    """
    min_max = np.array(CHANNEL_MIN_MAX[name]) # [2,]
    n_sweeps=x.shape[-1]
    
    # choose mean,var to get approximate [-1,1] scaling
    var=((min_max[1]-min_max[0])/2)**2 # scalar
    var=np.array(n_sweeps*[var,])    # [n_sweeps,]
    
    offset=(min_max[0]+min_max[1])/2    # scalar
    offset=np.array(n_sweeps*[offset,]) # [n_sweeps,]

    return keras.layers.Normalization(mean=offset,
                                      variance=var,
                                      name='Normalize_%s' % name)(x)

## Running the Model

Ok, we've got all of the TorNet model components figured out and imported (using the helper functions, etc.).
[Even getting that working turned out to be tricky - there were dependency issues to resolve, and while we'd 
like to just be able to import the functions rather than re-copying them here again, that wasn't working so well.]

In [6]:
model = build_model()

I0000 00:00:1731965103.208415   17130 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5564 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:01:00.0, compute capability: 8.6


In [7]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ DBZ (InputLayer)    │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ VEL (InputLayer)    │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ KDP (InputLayer)    │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ RHOHV (InputLayer)  │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ZDR (InputLayer)    │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ WIDTH (InputLayer)  │ (None, 120, 240,  │          0 │ -                 │
│                     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalize_DBZ       │ (None, 120, 240,  │          0 │ DBZ[0][0]         │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalize_VEL       │ (None, 120, 240,  │          0 │ VEL[0][0]         │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalize_KDP       │ (None, 120, 240,  │          0 │ KDP[0][0]         │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalize_RHOHV     │ (None, 120, 240,  │          0 │ RHOHV[0][0]       │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalize_ZDR       │ (None, 120, 240,  │          0 │ ZDR[0][0]         │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalize_WIDTH     │ (None, 120, 240,  │          0 │ WIDTH[0][0]       │
│ (Normalization)     │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Concatenate1        │ (None, 120, 240,  │          0 │ Normalize_DBZ[0]… │
│ (Concatenate)       │ 12)               │            │ Normalize_VEL[0]… │
│                     │                   │            │ Normalize_KDP[0]… │
│                     │                   │            │ Normalize_RHOHV[… │
│                     │                   │            │ Normalize_ZDR[0]… │
│                     │                   │            │ Normalize_WIDTH[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ isnan (Isnan)       │ (None, 120, 240,  │          0 │ Concatenate1[0][… │
│                     │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ where (Where)       │ (None, 120, 240,  │          0 │ isnan[0][0],      │
│                     │ 12)               │            │ Concatenate1[0][… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 8,084,225 (30.84 MB)

 Trainable params: 8,084,225 (30.84 MB)

 Non-trainable params: 0 (0.00 B)

Again, as mentioned above, of note here is the learning rate chosen, which is 10^-4.

The original version of this notebook attempted to use 10^-3, but that caused the model to fail to converge after just one epoch of training (later epochs failed to make any progress). In comparison, using 10^-4 (while slower initially) seems to work well in training the data over the course of at least four epochs.

As noted in the previous notebook, we might consider decaying the learning rate even further (due to our small batch size) later in this training for later epochs, because the small batch size can cause noisiness in gradient descent updates (so dampening those updates a bit can help the model converge). We'll monitor and see how that goes.

In [10]:
opt = keras.optimizers.Adam(learning_rate=1e-4)
loss = keras.losses.BinaryCrossentropy(from_logits=True)
model.compile(loss=loss, optimizer=opt)

We create a checkpoint saving function just to deal with iPython's many kernel crashes.
After each pass through the data and all batches, we'll save the weights, reload the model, and keep going.

We'll save the weights in `checkpoints/epoch_{number_of_epoch}.weights.h5`

In [11]:
def checkpoint_creator(checkpoint_path):
    # saving the model's weights in case the iPython kernel crashes (which it likes to do)
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                       save_weights_only=True,
                                       verbose=1)
    return cp_callback

def checkpoint_loader(checkpoint_path, model):
    model.load_weights(checkpoint_path)

First pass through the data. Again, with each pass, we set it up with:
- A batch_size of 64 (to fit the data into GPU memory)
- Shuffle the data with a random_state (we pick a new seed each time, but record that seed here so we don't reuse it in a later epoch)
- Set up a new checkpoint for each epoch (in case any one epoch of training crashes/fails).

In [12]:
preprocessed = make_tf_with_flips_loader(data_root = TORNET_DATA_INPUT_FOLDER, 
                                         data_type = "train", # or 'test'
                                         years = list(range(2013, 2023)),
                                         batch_size = 64, 
                                         weights = None,
                                         include_az = False,
                                         random_state = 5678,
                                         select_keys = ALL_VARIABLES + ["coordinates", "range_folded_mask"],
                                         tilt_last = True,
                                         from_tfds = False,
                                         tfds_data_version ="1.1.0")

In [ ]:
model.fit(preprocessed, epochs=1, callbacks=[checkpoint_creator("checkpoints/epoch_flipping_1_1e-4.weights.h5")])

I0000 00:00:1731973690.907013   17376 service.cc:148] XLA service 0x7f62f8019090 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731973690.907543   17376 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-11-18 15:48:11.049961: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1731973691.398709   17376 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1731973695.090274   17376 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
I0000 00:00:1731973715.098554   17376 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


   2256/Unknown 7306s 3s/step - loss: 1.4560

Now we loop through a bunch of epochs to train the CNN further, making sure to change the random_state each time to mini-batch the data differently each time.

In [ ]:
for epoch_num in range(2, 5):
    resampled = make_tf_with_flips_loader(data_root = TORNET_DATA_INPUT_FOLDER, 
                                          data_type = "train", # or 'test'
                                          years = list(range(2013, 2023)),
                                          batch_size = 64, 
                                          weights = None,
                                          include_az = False,
                                          random_state = 1234 + epoch_num,
                                          select_keys = ALL_VARIABLES + ["coordinates", "range_folded_mask"],
                                          tilt_last = True,
                                          from_tfds = False,
                                          tfds_data_version ="1.1.0")
    checkpoint_loader("checkpoints/epoch_flipping_{}_1e-4.weights.h5".format(str(epoch_num - 1)), model)
    model.fit(resampled, epochs=1, callbacks=[checkpoint_creator("checkpoints/epoch_flipping_{}_1e-4.weights.h5".format(epoch_num))])

## Testing on test data

In our test data, we DON'T want to do any flipping or data augmentation. So, before we run the test data 
evaluation, we have to bring back TorNet's original functions for loading the test data into a 
Tensorflow dataset.

After we do that, we can run through the testing data for each epoch to see how the model performed
after each epoch of training in terms of AUC.

In [10]:
def create_tf_dataset(files:str,
                      variables: List[str]=ALL_VARIABLES,
                      n_frames:int=1,
                      tilt_last: bool=True) -> tf.data.Dataset:
    """
    This is Tornet's main function for loading the data from the folder where it's all stored.
    
    As they stated, this creates a TF dataset object via the function read_file (which reads the NetCDF files
    into the data one at a time).
    """
    assert len(files)>0
    # grab one file to gets keys, shapes, etc
    data = read_file(files[0],variables=variables,n_frames=n_frames, tilt_last=tilt_last)
    
    output_signature = { k:tf.TensorSpec(shape=data[k].shape,dtype=data[k].dtype,name=k) for k in data }
    def gen():
        for f in files:
            yield read_file(f,variables=variables,n_frames=n_frames, tilt_last=tilt_last)
    ds = tf.data.Dataset.from_generator(gen,
                                        output_signature=output_signature)
    return ds

def make_tf_loader(data_root: str, 
            data_type:str='train', # or 'test'
            years: list=list(range(2013,2023)),
            batch_size: int=128, 
            weights: Dict=None,
            include_az: bool=False,
            random_state:int=1234,
            select_keys: list=None,
            tilt_last: bool=True,
            from_tfds: bool=False,
            tfds_data_version: str='1.1.0',
            num_epochs: int=3):
    """
    This TorNet library function is used to load the data into Tensorflow.
    We're going to use the `create_tf_dataset` function from above, 
    then we'll use `preproc` to preprocess it.

    One important note - we tried a bunch of different functions for shuffling 
    and batching, repeating the dataset, etc. to try to be able to run
    many epochs with one function call. It wasn't working.
    Even the `drop_remainder=True` that we've added here to ds.batch
    seems to not really have an effect, as the model training
    still throws an error at the end of training about running out of data.
    """
    
    if from_tfds: # fast loader
        import tensorflow_datasets as tfds
        import tornet.data.tfds.tornet.tornet_dataset_builder # registers 'tornet'
        ds = tfds.load('tornet:%s' % tfds_data_version ,split='+'.join(['%s-%d' % (data_type,y) for y in years]))
        # Assumes data was saved with tilt_last=True and converts it to tilt_last=False
        if not tilt_last:
            ds = ds.map(lambda d: pp.permute_dims(d,(0,3,1,2), backend=tf))
    else: # Load directly from netcdf files
        file_list = query_catalog(data_root, data_type, years, random_state)
        ds = create_tf_dataset(file_list,variables=ALL_VARIABLES,n_frames=1, tilt_last=tilt_last) 

    ds = preproc(ds,weights,include_az,select_keys,tilt_last)
    ds = ds.prefetch(tf.data.AUTOTUNE)

    # this has been adjusted to include drop_remainder=True
    ds = ds.batch(batch_size, drop_remainder=True)
    return ds

For each epoch that we trained through, let's test the model and see how it did after each pass of the data.
AUC is our evaluation metric here.

In [ ]:
test_data = make_tf_loader(data_root = TORNET_DATA_INPUT_FOLDER, 
                              data_type = "test",
                              years = list(range(2013, 2023)),
                              batch_size = 64, 
                              weights = None,
                              include_az = False,
                              random_state = 5678,
                              select_keys = ALL_VARIABLES + ["coordinates", "range_folded_mask"],
                              tilt_last = True,
                              from_tfds = False,
                              tfds_data_version ="1.1.0")

for epoch_num in range(1, 4):
    metrics = [keras.metrics.AUC(from_logits=True,name='AUC')]
    checkpoint_loader("checkpoints/epoch_flipping_{}.weights.h5".format(str(epoch_num)), model)
    model.compile(loss=loss, metrics=metrics)
    model.evaluate(test_data)

/usr/local/lib/python3.9/dist-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 54 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
I0000 00:00:1731917373.302069    9097 service.cc:148] XLA service 0x7f16d8003a80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731917373.302399    9097 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-11-18 00:09:33.336630: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1731917373.422577    9097 cuda_dnn.cc:529] Loaded cuDNN version 90300


      2/Unknown 10s 61ms/step - AUC: 0.3504 - loss: 0.1487   

I0000 00:00:1731917379.526763    9097 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


491/491 ━━━━━━━━━━━━━━━━━━━━ 1558s 3s/step - AUC: 0.6166 - loss: 0.2446


2024-11-18 00:35:27.138112: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-11-18 00:35:27.138189: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_16]]
2024-11-18 00:35:27.138202: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 7982527026973230273
2024-11-18 00:35:27.138209: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 4580209453287121430
2024-11-18 00:35:27.138212: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 16141768603555529616
2024-11-18 00:35:27.138216: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 14111782021850010702
2024-11-

491/491 ━━━━━━━━━━━━━━━━━━━━ 1529s 3s/step - AUC: 0.6162 - loss: 0.2632


2024-11-18 01:00:56.358591: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_16]]
2024-11-18 01:00:56.358634: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 7982527026973230273
2024-11-18 01:00:56.358645: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 4580209453287121430
2024-11-18 01:00:56.358649: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 16141768603555529616
2024-11-18 01:00:56.358653: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 14111782021850010702
2024-11-18 01:00:56.358657: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 15603253188103886432
2024-11-18 01:00:56.358661: I tensorflow/c

    139/Unknown 432s 3s/step - AUC: 0.6238 - loss: 0.2494